In [174]:
#In this notebook I want to investigate the technique of "phase mapping" (working name)

#Phase mapping takes the idea of the (Feynman) Path integral and tries to apply it to noisy data (intended for statistical noise); it can perhaps be seen as a method of error mitigation or as a method to improve the signal2noise ratio


#The main idea is the following: The big difference between noise and our signal is that the signal is a stable source of counts,
#while noise should create counts in a more statisticaö way

#Therefore the count yield of the noise is fluctuating


#In most physics experiments, the advantage of averaging is that stochastic noise can cancel itself bc it goes into both "directions" (negative and positive)

#In the case of count distributions though, it is always positive: it always adds up
#So we need a method that can make noisy signals cancel each other
#In nature, this is achieved by the path integral

#The idea that each possible realization of systems in nature has an Amplitude which is directly linked to its probability
#The amplitude is (more or less) given by the path integral A=\int e^{iS/\hbar}
#We see a phase factor with iS/\hbar as exponent

#This phase factor results in the action S being effectively minimized in nature
#the scale against which S is measured (\hbar) is the smalled sensible unit of action and incredibly small
#meaning on paths, for which S is not minimal, the phase factor will take all possible phases. Adding them up results in destructive interference
#For a stable path (meaning local extreme point, e.g. minimum), S is not fluctuating in first order. The phase factor will be roughly (or close to) one, resulting in constructive interference


#We want to apply this technique to our signal now
#We run a given circuit many times with a stochastic noise model

#the noisy part of the outcome will vary, while the signal will stay stable
#by making all outcomes interfere according to their fluctuations, we will weaken fluctuating signals and amplify stable signals


In [175]:
from qiskit import *
import numpy as np
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error

In [176]:
#we will run a 3 Qubit GHZ state as test
qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0,[1,2])

qc.measure_all()

In [177]:
#here we can change the sensitivity of the method
#it makes sense keeping it as 1 right now, but after having gone through the nb, one can play a bit with it, e.g. trying sensitivity=2

#I have already played a little bit with this parameter and choosing it bigger can actually yield a much better result
sensitivity=1

In [178]:
#choose our simulator
sim = Aer.get_backend("aer_simulator")

In [179]:
#the perfect result in this case is
perfect_result={"000": 0.5,
               "111": 0.5
               }

In [180]:
#next, we need our stochastic noise model

def get_noise(p_meas,p_gate):
    error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    error_gate1 = depolarizing_error(p_gate, 1)
    error_gate2 = error_gate1.tensor(error_gate1)

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure") # measurement error is applied to measurements
    noise_model.add_all_qubit_quantum_error(error_gate1, ["h"]) # single qubit gate error is applied to x gates
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"]) # two qubit gate error is applied to cx gates
        
    return noise_model


noise_model=get_noise(0.05,0.05)
#5 percent probability for each error

In [181]:
#prepare circuit for running
trans_qc=transpile(qc,sim)
qobj=assemble(trans_qc)

In [182]:
#run qc multiple times
dictionary_list=[]
for i in range(100):
    dictionary_list.append(sim.run(qobj, noise_model=noise_model).result().get_counts())

In [183]:
#look at the mean fidelities of each of the outcomes
from qiskit.quantum_info import hellinger_fidelity
fid=[]
for dictionary in dictionary_list:
    fid.append(hellinger_fidelity(dictionary,perfect_result))

In [184]:
#look at the mean fidelity
print(f" the mean fidelity is given by {np.mean(fid)}")

 the mean fidelity is given by 0.7803690511085715


In [185]:
#get dictionary with relative mean counts for each outcome  (these will serve as reference scales for exponentials)
key_list=dictionary_list[0].keys()


#get mean values
mean_fid={}
for key in key_list:
    mean_fid[key]=0
    
    
#fill up the array
for dictionary in dictionary_list:
    for key in dictionary:
        mean_fid[key]+=dictionary[key]
for key in mean_fid:
    mean_fid[key]/=len(dictionary_list)

In [186]:
#now apply the phase mapping technique (apply the phase weights to each outcome of each single measurement)

total_dict_phase={}
for key in key_list:
    total_dict_phase[key]=0

for dictionary in dictionary_list:
    for key in total_dict_phase:
        if key in dictionary:
            total_dict_phase[key]+=dictionary[key]*np.exp(1j*2*np.pi*dictionary[key]*sensitivity/mean_fid[key])

In [187]:
#look at dictionary before we take absolute values
print(total_dict_phase)



{'000': (38702.0997317676+382.5468657027069j), '111': (39033.06473349433+298.0774513569704j), '101': (2895.219552036939+319.6503186522453j), '110': (2024.157983882905+265.94268376523195j), '010': (2741.3587166036327+304.50039043258994j), '011': (2077.3966503310953+303.1282113547756j), '001': (1625.133520763893+238.2923831371981j), '100': (1885.646788291754+272.2216329522528j)}


In [188]:
#for an actual result, we take the absolute value of each outcome in the dictionary

In [189]:
for key in total_dict_phase:
    total_dict_phase[key]=np.abs(total_dict_phase[key])

In [190]:
#we look at fidelity of our distribution obtained by apply the phase mapping procedure
fid_ph1=hellinger_fidelity(total_dict_phase,perfect_result)
print(fid_ph1)

0.85334415506561


In [191]:
#what we have done was to sum up our counts/results for each outcome weighted by a phase factor

#another idea had was to sum up the phase factors directly and multiply the outcomes of an averaged outcome dictionary by the respective aplitudes

Amplitudes={}

for key in key_list:
    Amplitudes[key]=0


for key in Amplitudes:
    for i in range(len(dictionary_list)):
        if key in dictionary_list[i].keys():
            Amplitudes[key]+=np.exp(1j*2*np.pi*dictionary_list[i][key]/(mean_fid[key]))
print(f" complex Amplitudes are {Amplitudes}")

#take the absolute values of each entry of Amplitudes
for key in Amplitudes:
    Amplitudes[key]=np.abs(Amplitudes[key])
    
print(f"absolute values of Amplitudes are {Amplitudes}")

 complex Amplitudes are {'000': (97.10459890515628+0.05259988014798969j), '111': (97.40500788175277-0.07252715257588993j), '101': (67.71930602433719-1.212866882619275j), '110': (60.51041111650118-2.1728330397684j), '010': (63.59180291276981-2.2432443194216862j), '011': (59.382081729350254-1.0581578181047095j), '001': (49.809581479517675-4.0553170389813475j), '100': (54.262969273456775-2.8813814107050026j)}
absolute values of Amplitudes are {'000': 97.10461315137746, '111': 97.40503488337849, '101': 67.7301665027688, '110': 60.54941004590135, '010': 63.63135660013211, '011': 59.39150889209037, '001': 49.97439347757371, '100': 54.339416570350394}


In [192]:
#next we obtain the total averaged outcome dictionary
dict_total={}
for key in key_list:
    dict_total[key]=0
#add up all the result
for dictionary in dictionary_list:
    for key in dictionary:
        dict_total[key]+=dictionary[key]

In [193]:
#now we multiply the entries by our respective amplitudes

for key in dict_total:
    dict_total[key]*=Amplitudes[key]

In [194]:
#check the fidelity
fid_ph2=hellinger_fidelity(perfect_result,dict_total)
print(fid_ph2)

#the fidelity in this case is also better than of the single measurements

0.8526801535932208


In [195]:
#we have seen that we can improve the fidelity of our measurements by using a superposition of different mappings
#weighted by local phases (local meaning the phase depends on the outcome)

#this achieves that noisy outcomes get suppressed (as can be seen by the Amplitude dictionary)
#effectively, we improve the signal to noise ratio

#we could optimize this surely in many ways. One possibility that I can directly think of would be to try different reference scales in the complex exponential, 
#so that we actually get proper destructive interference for the noise
#by choosing our reference scale smaller, we make the method more sensitive towards noise (thus more unstable but potentially more effective)

In [196]:
#again, to compare the numbers:
print(f"the average fidelity by single circuit runs is {np.mean(fid)}")
print(f"the best fidelity by single cirucit runs is {fid[np.argmax(fid)]}")

the average fidelity by single circuit runs is 0.7803690511085715
the best fidelity by single cirucit runs is 0.8065223797608921


In [197]:
print(f"the fidelities using the phase mapping technique are {fid_ph1} and {fid_ph2}" )

the fidelities using the phase mapping technique are 0.85334415506561 and 0.8526801535932208


In [ ]:
#one can further explore the effects of choosing different reference scales, e.g. trying 0.5*mean_fid instead of mean_fid